In [1]:
import pandas as pd
import time
import calendar
import ccxt
import os
import numpy as np
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST 
from alpaca_trade_api.rest import TimeFrame
from alpaca_trade_api.rest import TimeFrameUnit
import random
import krakenex
from pykrakenapi import KrakenAPI
from datetime import datetime
import questionary
import requests
import json
import functions

In [2]:
load_dotenv()

ALPACA_API_KEY_ID = os.getenv('ALPACA_API_KEY_ID')
ALPACA_API_SECRET_KEY = os.getenv('ALPACA_API_SECRET_KEY')

KRAKEN_API_KEY_ID =  os.getenv('KRAKEN_API_KEY_ID')
KRAKEN_API_SECRET_KEY = os.getenv('KRAKEN_API_SECRET_KEY')

In [89]:
#get crypto data
crypto_OHLC_df = {}
def getCryptoOHLCData(tickers):
   
    headers = {'X-CoinAPI-Key' : 'API Key: DABB5DFC-447C-4F57-8600-147722F5C7BA'}
    
    for ticker in tickers:
        #url = f'https://rest.coinapi.io/v1/exchangerate/{ticker}/USD/history?period_id=1DAY&time_start=2019-01-01T00:00:00&time_end=2022-02-01T00:00:00&limit=900'
        url = f'https://rest.coinapi.io/v1/ohlcv/BITSTAMP_SPOT_{ticker}_USD/history?period_id=1DAY&time_start=2019-01-01T00:00:00&time_end=2022-02-01T00:00:00&limit=900'

        response = requests.get(url, headers=headers)
        print(response.headers)
        data = json.loads(response.content)
        
        df = pd.DataFrame(data)

        ticker_ohlc = df
        ticker_ohlc['symbol'] = ticker
        crypto_OHLC_df[ticker] = ticker_ohlc.drop(['time_period_start','time_open','time_close','price_open','price_high','price_low'], axis=1).reset_index()
        time.sleep(1)
    return crypto_OHLC_df

In [90]:


tickers = ['BTC','ETH','LUNA','SOL','XRP','ADA','AVAX','DOT','DOGE','SHIB','MATIC', 'DAI','LTC','ATOM','LINK','UNI','TRX','BCH']

data = getCryptoOHLCData(tickers)

{'date': 'Mon, 11 Apr 2022 20:30:58 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'x-concurrencylimit-limit': '10', 'x-concurrencylimit-remaining': '10', 'x-ratelimit-used': '1077', 'x-ratelimit-overage': 'disabled', 'x-ratelimit-limit': '1000', 'x-ratelimit-remaining': '-77', 'x-ratelimit-reset': '2022-04-12T20:30:58.1917217Z'}


ValueError: If using all scalar values, you must pass an index

In [80]:
crypto_tickers = ['BTC','ETH','LUNA','SOL','XRP','ADA','AVAX','DOT','DOGE','SHIB','MATIC', 'DAI','LTC','ATOM','LINK','UNI','TRX','BCH']


all_ohlc = pd.concat(data.values(), join="inner")#.reset_index().set_index("time_close").pivot(columns = 'symbol').dropna()
all_ohlc['time_period_end'] = pd.to_datetime(all_ohlc['time_period_end'])
all_ohlc['time_period_end'] = all_ohlc['time_period_end'].dt.strftime('%Y/%m/%d')
all_ohlc = all_ohlc.drop(['index'], axis=1).set_index('time_period_end').pivot(columns = 'symbol')
all_ohlc

rate_close                                                     \
symbol                 ADA ATOM       AVAX         BCH          BTC       DAI   
time_period_end                                                                 
2019/01/02        0.042172  NaN        NaN  162.580906  3813.724043  0.998083   
2019/01/03        0.044704  NaN        NaN  169.921819  3888.637329  0.998687   
2019/01/04        0.042234  NaN        NaN  159.588340  3785.973167  0.996296   
2019/01/05        0.043491  NaN        NaN  159.442268  3826.806638  0.996097   
2019/01/06        0.044286  NaN        NaN  157.992761  3794.363658  0.997334   
...                    ...  ...        ...         ...          ...       ...   
2022/01/29             NaN  NaN  67.135768         NaN          NaN       NaN   
2022/01/30             NaN  NaN  72.142264         NaN          NaN       NaN   
2022/01/31             NaN  NaN  67.947271         NaN          NaN       NaN   
2022/02/01             NaN  NaN  70.055389         NaN          NaN       NaN   
2022/02/02             NaN  NaN  69.914505         NaN          NaN       NaN   

                                                                      \
symbol               DOGE       DOT         ETH      LINK        LTC   
time_period_end                                                        
2019/01/02       0.002365  0.005887  139.420219  0.296860  31.612903   
2019/01/03       0.002354  0.006154  153.291344  0.312677  32.950778   
2019/01/04       0.002294  0.006095  147.032941  0.362437  31.562766   
2019/01/05       0.002270  0.006008  153.363119  0.436739  32.084004   
2019/01/06       0.002258  0.005879  153.846434  0.390293  34.513264   
...                   ...       ...         ...       ...        ...   
2022/01/29            NaN       NaN         NaN       NaN        NaN   
2022/01/30            NaN       NaN         NaN       NaN        NaN   
2022/01/31            NaN       NaN         NaN       NaN        NaN   
2022/02/01            NaN       NaN         NaN       NaN        NaN   
2022/02/02            NaN       NaN         NaN       NaN        NaN   

                                                                             
symbol               LUNA MATIC      SHIB       SOL       TRX UNI       XRP  
time_period_end                                                              
2019/01/02       0.002021   NaN       NaN  0.001203  0.019321 NaN  0.362180  
2019/01/03       0.002021   NaN       NaN  0.001329  0.019902 NaN  0.371462  
2019/01/04       0.002021   NaN       NaN  0.001289  0.019760 NaN  0.354374  
2019/01/05       0.002021   NaN       NaN  0.001342  0.021265 NaN  0.356697  
2019/01/06       0.001340   NaN       NaN  0.001334  0.022036 NaN  0.349480  
...                   ...   ...       ...       ...       ...  ..       ...  
2022/01/29            NaN   NaN  0.000021       NaN       NaN NaN       NaN  
2022/01/30            NaN   NaN  0.000022       NaN       NaN NaN       NaN  
2022/01/31            NaN   NaN  0.000021       NaN       NaN NaN       NaN  
2022/02/01            NaN   NaN  0.000021       NaN       NaN NaN       NaN  
2022/02/02            NaN   NaN  0.000021       NaN       NaN NaN       NaN  

[1128 rows x 18 columns]

In [40]:
#Kraken Direct Api

kraken_api = krakenex.API(KRAKEN_API_KEY_ID, KRAKEN_API_SECRET_KEY)

#print(kraken_api.query_public("AssetPairs")["result"])

crypto_tickers = ['BTC','ETH','LUNA','SOL','XRP','ADA',
    'AVAX','DOT','DOGE','SHIB','MATIC',
    'DAI','LTC','ATOM','LINK','UNI','TRX','BCH']

k = KrakenAPI(kraken_api)

all_ohlc = pd.DataFrame()

dataframes = {}

date =  datetime.strptime('01/01/2019', '%m/%d/%Y')
utc_time = calendar.timegm(date.utctimetuple())

for ticker in crypto_tickers:
    ticker_ohlc, last = k.get_ohlc_data((ticker + 'USD'),10080,1548111600)
    ticker_ohlc['symbol'] = ticker
    dataframes[ticker] = ticker_ohlc.drop(['open', 'high', 'low', 'volume', 'vwap', 'count', 'time'], axis=1).reset_index()
    time.sleep(1)

In [41]:
print(utc_time)

all_ohlc = pd.concat(dataframes.values(), join="inner").set_index("dtime").pivot(columns = 'symbol').dropna()
all_ohlc.columns = crypto_tickers
all_ohlc



1546300800


,BTC,ETH,LUNA,SOL,XRP,ADA,AVAX,DOT,DOGE,SHIB,MATIC,DAI,LTC,ATOM,LINK,UNI,TRX,BCH
dtime,,,,,,,,,,,,,,,,,,
2021-12-16,1.326544,27.2838,116.97,438.28,48624.9,0.99990,0.173188,27.2159,3980.44,19.90989,155.41,85.43,2.5045,0.000035,177.58,0.079046,16.223,0.95459
2021-12-23,1.331084,27.9400,102.98,429.95,46473.2,1.00000,0.167974,26.8053,3628.29,19.72420,145.48,84.32,2.4755,0.000034,170.54,0.076395,17.115,0.81704
2021-12-30,1.229072,38.6326,97.46,399.37,43410.0,0.99990,0.158936,26.6691,3538.91,25.22049,135.29,78.41,2.1804,0.000030,154.38,0.071153,17.084,0.77412
2022-01-06,1.313360,40.2254,95.95,383.81,43927.8,1.00000,0.162908,27.3178,3373.77,26.75901,141.91,81.93,2.3964,0.000031,151.56,0.068101,16.267,0.79808
2022-01-13,1.336730,37.1625,82.62,369.06,41686.2,1.00010,0.161705,24.0900,3084.31,21.56829,136.26,81.27,2.0312,0.000027,135.35,0.069192,15.135,0.73848
2022-01-20,1.077562,32.3498,65.87,289.67,36827.6,1.00006,0.143059,18.0890,2462.96,15.26000,107.07,58.80,1.5893,0.000021,92.07,0.056164,10.571,0.61978
2022-01-27,1.027207,26.3987,67.84,278.36,36903.3,0.99999,0.137104,18.8014,2682.36,15.85526,108.38,47.60,1.5417,0.000021,101.95,0.058639,10.513,0.60143
2022-02-03,1.195116,30.9500,90.13,346.42,44420.0,1.00007,0.159048,21.9630,3246.10,18.67783,140.16,57.56,2.0387,0.000033,114.10,0.069943,12.329,0.87094
2022-02-10,1.083038,28.4978,95.20,334.95,43873.1,1.00010,0.148737,19.7618,3124.43,17.14156,127.66,56.02,1.8392,0.000031,101.90,0.066472,11.232,0.83786


In [19]:
all_ohlc.columns = crypto_tickers
all_ohlc_pct = all_ohlc.pct_change()

In [23]:
# Random Crypto
def random_portfolios_generator(tickers, n):
    random_portfolios = pd.DataFrame(tickers, columns = ['ticker'])
    for n in range(0, n):
        random_portfolios[f'random weights {n+1}'] = [np.random.rand() for i in range (0, len(tickers))]
        random_portfolios[f'random weights {n+1}'] = random_portfolios[f'random weights {n+1}'] / sum(random_portfolios[f'random weights {n+1}'])
    return(random_portfolios)

random_crypto_portfolio = pd.Series()


random_crypto = random_portfolios_generator(crypto_tickers, 10)

for n in range(len(all_ohlc)):
    for m in range(len(random_crypto)):
        random_crypto_portfolio.append(all_ohlc.iloc(n+1).dot(random_crypto.loc(m+1)))

# random_crypto

C:\Users\kenne\AppData\Local\Temp\ipykernel_18092\3682669130.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  random_crypto_portfolio = pd.Series()


AttributeError: '_LocIndexer' object has no attribute 'dot'

In [15]:
# Daily Returns

random_crypto_daily_returns = all_ohlc.pct_change().dot(list(random_crypto.loc[:, 'random weights 1']))
random_crypto_daily_returns



dtime
2022-03-08         NaN
2022-03-09    0.054815
2022-03-10   -0.038774
2022-03-11   -0.011147
2022-03-12   -0.001264
2022-03-13   -0.023852
2022-03-14    0.028037
2022-03-15   -0.000804
2022-03-16    0.042211
2022-03-17   -0.001342
2022-03-18    0.024882
2022-03-19    0.027936
2022-03-20   -0.027857
2022-03-21    0.011220
2022-03-22    0.024762
2022-03-23    0.021967
2022-03-24    0.023598
2022-03-25   -0.022464
2022-03-26    0.015451
2022-03-27    0.044810
2022-03-28   -0.004456
2022-03-29    0.025349
2022-03-30    0.011373
2022-03-31   -0.024578
2022-04-01    0.019632
2022-04-02    0.001246
2022-04-03    0.019152
2022-04-04   -0.013852
2022-04-05   -0.019427
2022-04-06   -0.079971
2022-04-07    0.023057
2022-04-08   -0.040642
2022-04-09    0.007177
dtype: float64

In [ ]:

# kraken = ccxt.kraken({
#     'apiKey': KRAKEN_API_KEY_ID,
#     'secret': KRAKEN_API_SECRET_KEY,
# })

# markets = kraken.load_markets()

# print(kraken)
# if (kraken.has['fetchTickers']):
#     print(kraken.fetch_tickers(['BTC/USD','ETH/USD','LUNA/USD','SOL/USD','XRP/USD','ADA/USD',
#     'AVAX/USD','DOT/USD','DOGE/USD','SHIB/USD','MATIC/USD',
#     'DAI/USD','LTC/USD','ATOM/USD','LINK/USD','UNI/USD','TRX/USD','BCH/USD'])) 

SyntaxError: unexpected EOF while parsing (3957576514.py, line 12)

In [ ]:
#Alpaca 

alpaca_api = REST()

#Facebook, Amazon, Apple, Netflix, Google, Microsoft, Uber, Lyft, Airbnb

FANNGMULA_tickers = ["META", "AMZN", "AAPL", "NTFX", "GOOG", "MSFT", "UBER", "LYFT", "ABNB"]

alpaca_api.get_bars(FANNGMULA_tickers, TimeFrame(45, TimeFrameUnit.Minute), "2021-06-08", "2021-06-08", adjustment='raw').df

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-06-08 07:30:00+00:00,126.10,126.16,125.9600,126.06,20951,304,126.049447,AAPL
2021-06-08 08:15:00+00:00,126.05,126.30,126.0500,126.30,21181,349,126.231904,AAPL
2021-06-08 09:00:00+00:00,126.27,126.32,126.2200,126.28,15955,308,126.284120,AAPL
2021-06-08 09:45:00+00:00,126.29,126.40,125.9000,125.90,30179,582,126.196877,AAPL
2021-06-08 10:30:00+00:00,125.90,126.75,125.8400,126.75,105380,1376,126.530863,AAPL
...,...,...,...,...,...,...,...,...
2021-06-08 20:15:00+00:00,49.81,49.88,49.7800,49.86,21235,97,49.860162,UBER
2021-06-08 21:00:00+00:00,49.86,49.88,49.8600,49.86,44362,30,49.879444,UBER
2021-06-08 21:45:00+00:00,49.86,49.88,49.8000,49.85,1739,12,49.864410,UBER
